In [1]:
from src.capstone.evaluation import Evaluator
from src.capstone.cbf import CBF
from src.capstone.settings import Env
from src.capstone.noise import DoubleIntegratorEnv

from bound_propagation.parallel import Parallel
from bound_propagation.bivariate import Mul
from bound_propagation.reshape import Select
from bound_propagation.polynomial import UnivariateMonomial, Pow
from bound_propagation.linear import FixedLinear
from bound_propagation.activation import Sin

import numpy as np
import gymnasium as gym

import torch
import torch.nn as nn

import math



/Users/wesselwolterbeekmuller/Documents/Documents - Wessel’s MacBook Pro/Studie/Engineering with AI/Capstone/SCBF Paper/Capstone/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class NNDM(nn.Module):
    # input [x pos, y pos, x speed, y speed, ux, uy]
    
    def __init__(self):
        super(NNDM, self).__init__()
        self.dt = 0.05
        self.layer1 = nn.Linear(6, 4, bias=False)
        self.weights = torch.tensor([[1., 0., self.dt, 0., ((self.dt**2)/2), 0], 
                                     [0., 1., 0., self.dt, 0., ((self.dt**2)/2)], 
                                     [0., 0., 1., 0., self.dt, 0.], 
                                     [0., 0., 0., 1., 0., self.dt]])
        self.layer1.weight = nn.Parameter(self.weights)

    def forward(self, x):
        return self.layer1(x)

nndm = NNDM()



In [3]:
class Agent(nn.Module):
    # dummy agent of linear form a = c1 * s1 + c2 * s2 + c3, always outputs 0.
    
    def __init__(self):
        super(Agent, self).__init__()
        
        self.layer = nn.Linear(2, 2)
        
        # dummy output of u=0
        self.layer.weight = nn.Parameter(torch.tensor([[0., 0., 0., 0.], [0., 0., 0., 0.]]))
        self.layer.bias = nn.Parameter(torch.tensor([50., 0.]))
    
    def forward(self, x):
        return self.layer(x)
    
    def select_action(self, x, exploration=False):
        if exploration:
            raise ValueError('This model is not implemented for exploration')
        else:
            return self.forward(x)

In [4]:
policy = Agent()


In [5]:
class DoubleIntegrator(Env):
    def __init__(self):
        self.is_discrete = False
        
        self.settings = {
            'noise': [],
            'max_frames': 100
        }
        
        self.h_function = nn.Sequential(
            FixedLinear(
                torch.tensor([
                    [1., 0, 0, 0],
                    [0, 1, 0, 0]
                ]),
                torch.tensor([0., 0.])
            ),
            Pow(2),
            FixedLinear(
                torch.tensor([
                    [-1 / 0.5 ** 2, 0],
                    [0, -1 / 0.5 ** 2]
                ]),
                torch.tensor([1., 1.])
            )
        )

        self.h_ids = [0, 4]
        self.std = [0.0, 0.0]
        self.env = DoubleIntegratorEnv()

In [6]:
env = DoubleIntegrator()
print(env.env.observation_space.shape)

(4,)


In [7]:
cbf = CBF(env, nndm, policy,
          alpha=[0.9, 0.9],
          delta=[0., 0.],
          no_action_partitions=2,
          no_noise_partitions=2,
          stochastic=True)

NotImplementedError: Module type not supported - add BoundModule for layer to factory

In [ ]:
evaluator = Evaluator(env, cbf)
evaluator.plot(policy, n=100, seed=[0.1, 0.])

NameError: name 'cbf' is not defined